In [32]:
import tkinter as tk

In [33]:
class CurveGrid(tk.Canvas):
    def __init__(self, master, **kwargs):
        super().__init__(master, **kwargs)
        self.selected_index = None
        self.lim = 4
        self.x = 0
        self.y = 0
        self.start_x = self.end_y = 50
        self.end_x = self.start_y = 560
        self.start_point = (self.start_x, self.start_y)
        self.end_point = (self.end_x, self.end_y)

        # left button
        self.bind("<Button-1>", self.on_click)
        # click and hold
        self.bind("<B1-Motion>", self.on_drag)
        # left mouse button release
        self.bind("<ButtonRelease-1>", self.on_release)

        self.points = [
            self.start_point,
            self.start_point,
            (150, 200),
            (250, 100),
            self.end_point,
            self.end_point
        ]

        self.lines = []

        # create Label coords,
        self.label = tk.Label(
            master,
            text=f"x: 0\ny: 0",
            anchor="w",
            justify="left",
            bg="#333333",
            fg="#aaaaaa",
            font=("Arial", 14)
        )
        self.label.place(x=50, y=580)
        self.draw_grid()
        self.draw_line()


    def draw_grid(self):
        # vertical lines
        for x in range(50, 561, 102):
            self.create_line(x, 50, x, 561, fill="#444444")

        # horizontal lines,
        for y in range(50, 561, 102):
            self.create_line(50, y, 561, y, fill="#444444")


    def draw_line(self):
        self.delete("curve", "point")
     
        # curve
        self.lines = []
        for i in range(len(self.points) - 1):
            self.lines.append(
                self.create_line(
                    self.points[i],
                    self.points[i+1],
                    fill="#aaaaaa",
                    width=1,
                    tags="curve"
                )
            )
     
        # points
        for i, (x, y) in enumerate(self.points, start=0):
            if i not in (0, len(self.points)-1):
                self.create_oval(x-3, y-3, x+3, y+3, fill="#aaaaaa", outline="black", tags="point")


    def distance(self, i, x, y):
        return (self.points[i][0]-x)**2 + (self.points[i][1]-y)**2
     
     
    def on_click(self, event):
        self.x, self.y = event.x, event.y
     
        index = min(range(len(self.points)), key=lambda i: self.distance(i, self.x, self.y))
        self.selected_index = min(max(1, index), len(self.points)-2)
     
     
     
    # drag point,
    def on_drag(self, event):
     
        # restricted movement coordinates,
        self.x = max(self.start_x, min(event.x, self.end_x))
        self.y = max(self.end_y, min(event.y, self.start_y))
     
        # selected point,
        self.points[self.selected_index] = (self.x, self.y)
     
        # coord: x -> next point, previous point
        prev_x, _ = self.points[self.selected_index - 1]
        next_x, _ = self.points[self.selected_index + 1]
     
        def limited_point(formula):
            self.x = formula
            self.points[self.selected_index] = (formula, self.y)
     
        # limit min distance between points,
        if self.selected_index != 1 and self.x - prev_x < self.lim:
            limited_point(prev_x + self.lim)
        elif self.selected_index != len(self.points)-2 and next_x - self.x < self.lim:
            limited_point(next_x - self.lim)
     
        self.label.config(text=f"x: {round((self.x - 50) / 2)}\ny: {round((560 - self.y) / 2)}")
     
        # boundary invisible points
        if self.selected_index == 1:
            self.points[0] = (self.start_x, self.y)
        elif self.selected_index == len(self.points) - 2:
            self.points[-1] = (self.end_x, self.y)
     
     
        self.draw_line()
     
    def on_release(self, event):
        self.selected_index = None

In [34]:
root = tk.Tk()
canvas = CurveGrid(root, width=600, height=650, bg="#333333")
canvas.pack(expand=True, fill="both")
canvas.configure(bg="#333333")

root.title("сurves")
root.mainloop()